### INE 410146 - Applied Machine Learning
$\textbf{Author: Prof. Mateus Grellert}$



Bibliographic references used in this lesson:
- Zheng, Alice; Casari, Amanda. Feature engineering for machine learning: principles and techniques for data scientists." - Chapters 2 and 3
- Aggarwal, Charu C. Data Mining: The Textbook, Section 2.2 and Section 14.2
- Bramer, Max. Principles of data mining - Chapter 20

In [8]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import warnings

warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
warnings.filterwarnings("ignore", message="MatplotlibDeprecationWarning")


# 4 - Feature Extraction and Engineering

Sometimes our data set contains raw data that doesn't seem useful at first, even though we know it might contain important information. In most cases, this also translates to features that are not useful in our modeling process, leading to inefficient predictions. In such scenarios, we might be tempted to consider our data is just a bunch of garbage and redo the collection process (if that's even a possibility). However, we will see in this lesson that sometimes we must simply see our data differently, or apply some kind of transformation to make it useful.

The name of this process is called **Feature  Engineering**, which can be defined as the  act of extracting features from raw data and transforming them into formats that are suitable for machine learning models. According to Zheng and Casari in their <i>"Feature Engineering for Machine Learning"</i> book, this is a crucial step in the machine learning pipeline, because the right features can ease the difficulty of modeling, and therefore to better results. 

Many authors include the process of mapping raw information to **feacture vectors** (the input format of most machine learning algorithms) as part of the Feature Engineering step, while others call it **Feature Extraction** and discuss it separately. We will mix both of these terms in this lesson, understanding that this step involves all the preparations required to initiate the training step.

Feature engineering involves a lot of **domain knowledge** to obtain useful information from unprocessed data, so make sure you keep one or more specialists really close to you during this process. Since our lessons are not focused on a specific domain, in this lesson we will learn some tools and techniques that work in most cases. We will discuss the following data types separately:
- Numeric and categorical
- Images
- Text
- Time series

Let's start with the most basic data types and how we can improve them.

## 4.1 - Numeric and Categorical Data

Extracting information from numeric data is fairly easy, because we are already working with structured data that only require a bit of fine-tuning to make it more interesting for models. At the same time, the options are limited due to the fact that data is already represented as a scalar value. Let's see some basic techniques.

### 4.1.1 - One-Hot Encoding

Many algorithms require numeric to operate, even for categorical variables. Therefore, it is necessary to map the categories to numbers. For instance, if we have a ``color`` category with three possible values ``('red', 'green', 'blue')``, then we need to find a numeric mapping of each value to make this feature/class usable for training.

The naive approach involves mapping each category to an integer value, so our tuple ('red', 'green', 'blue') could be mapped to the values (1, 2, 3). This is sometimes referred to as **Label Encoding**. While this will solve the numeric data constraint, it also gives us an implicit notion of order or rank. The problem here is that some we are implictly injecting a notion of order to our variable, and this might lead to confusion in the analyses and inefficient modeling.

**Note:** Label Encoding wouldn't be a problem if our categorical values where ordinal, for instance ``temperature = ('low', 'mid', 'high')``. In these cases, mapping categories to integers that respect the order is more than welcome.

A better way to perform this mapping is using what is called a **One-Hot Encoding** of our variable. This technique comes from digital circuits and represents an array of bits where only a single value is high (hot). In our case, we map each category to an index in this array, and set this index to `1` when thiTherefore, our ``color`` category could be mapped to the following vectors:
 - `red` -> `[0, 0, 1]`
 - `green`-> `[0, 1, 0]`
 - `blue`-> `[1, 0, 0]`

### 4.1.2 - Quantization (Binning)

The process of **Quantization** involves mapping a larger set of values to a smaller one, usually with a finite number of elements. Quantization is widely used in digital circuits to represent analog/continuous data from the real world in a digital/discrete domain. 

In machine learning, quantization techniques can be used to reduce the effects of minor disparities in the data or to convert numeric variables to categorical ones. The first task can be performed with simple operations like **rounding** and **truncation**, but the second one involves using a technique called **binning**.

Binning (also called bucketing) is the process of converting a continuous feature into multiple gropus called bins or buckets, typically based on value range. A good example is the `age` attribute, which can be transformed to an ``age_group`` feature with ranges like 0-5 years old, 5-10 years old etc. Each bin represents a counter that is incremented whenever a sample falls within the associated interval. Another practical examples are the **histograms** (discussed in Lesson 2) make use of binning to represent the distribution of variables.

### 4.1.3 - Scaling, Normalization & Standardization

While not particularly a feature extraction method per se, the process of feature scaling is a crucial step that is **required** by many machine learning algorithms that rely on the magnitude of features to compute distances. This includes Neural Networks, Support Vector Machines, K-means Clustering to name a few. In addition, scaling can help avoiding overflow in subsequent computations. For more information, read [this blog post](https://medium.com/greyatom/why-how-and-when-to-scale-your-features-4b30ab09db5e) about when to scale your data.

Scaling is the process of mapping a range of values to another. Usually, the destination range is either `[-1;1]` or `[0;1]`. Different strategies can be used to achieve this:
 - Normalization (Min-max Scaling): this scaling method typically involves mapping features to the `[0;1]`, using the following equation: $x_s = \frac{x - x_{min}}{x_{max}-x_{min}}$
 - Standardization: a standardized series has a mean of 0 and a standard deviation of 1. To accomplish this, the following equation can be used $x_s = \frac{x - \mu}{\sigma}$, where $\mu$ and $\sigma$ respectively stand for the mean and standard deviation of the series $X$.
 
There are other scalers available, and there is no recipe for which one is the most efficient one. Some might be more beneficial for a specific type of data or training algorithm, so it is recommended that a few different scalars are included in your Feature Extraction pipeline. For instance, scalers based on [power transforms](https://en.wikipedia.org/wiki/Power_transform) like the Box-Cox transform and the Yeo-Johnson transform provide an automatic way of making your data more Gaussian like, which is useful for parametric statistics.

### 4.1.4 - Pairwise Features

It is also possible to combine two or more features to create a new one. This is usually very dependent on the domain, but in general terms we can apply basic arithmetic operations like addition, subtraction, multiplication and division to model such relationships. For instance, if we have the price per unit of a product in a sales data set and the number of units sold, we can compute the total amount of sales for each product. In the health sciences studies, it is common to model the interaction between a categorical and a numeric variable using a multiplication (read [this document](http://www.utstat.toronto.edu/~brunner/DataAnalysisText/Interactions.pdf) for a more detailed information).

For binary features, logical operations can model relationships like the presence of both events (AND), as well as the presence of at least one event (OR), and so on. 

### 4.1.5 - Python Example

Let's practice a bit using the [Adult income dataset](https://www.kaggle.com/wenruliu/adult-income-dataset). It consists of classifying interviewed people into groups who receive more and less then 50K dollars a year. Our data contains 6 continuous and 8 categorical attributes, one of which is a rank (education). Some attribute Information can be found in [this link](http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html), although it doesn't describe the meaning of each attribute.

The missing values in this dat set were marked with the `?` symbol, so we will tell pandas that they represent missing values with the `na_values` parameters.


In [160]:
import pandas as pd
import numpy as np 

df = pd.read_csv('../DATASETS/adult.csv', sep = ',', na_values = '?')
df['income'] = df['income'].map({'>50K': 1,'<=50K': 0})
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        46043 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       46033 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   47985 non-null  object
 14  income           48842 non-null  int64 
dtypes: int64(7), object(8)
memory usage: 5.6+ MB


We have several categorical and numerical columns, some of which have missing values. Let's check their values and see the counts of the top 4 categories to see if they are well distributed using the `value_counts()` method.

In [161]:
print('\nTop-4 value counts of each categorical column:')
for col in df.columns:
    if df[col].dtype == 'object':
        counts = df[col].value_counts()
        print(f'Total categories for {col}: {counts.shape[0]}')
        print(counts.head(4))
        print()



Top-4 value counts of each categorical column:
Total categories for workclass: 8
Private             33906
Self-emp-not-inc     3862
Local-gov            3136
State-gov            1981
Name: workclass, dtype: int64

Total categories for education: 16
HS-grad         15784
Some-college    10878
Bachelors        8025
Masters          2657
Name: education, dtype: int64

Total categories for marital-status: 7
Married-civ-spouse    22379
Never-married         16117
Divorced               6633
Separated              1530
Name: marital-status, dtype: int64

Total categories for occupation: 14
Prof-specialty     6172
Craft-repair       6112
Exec-managerial    6086
Adm-clerical       5611
Name: occupation, dtype: int64

Total categories for relationship: 6
Husband          19716
Not-in-family    12583
Own-child         7581
Unmarried         5125
Name: relationship, dtype: int64

Total categories for race: 5
White                 41762
Black                  4685
Asian-Pac-Islander     1519
Am

Now let's do some basic feature extraction and engineering in our columns:
 1) The `native-country` attribute has 41 categories, but most of the people are from the US. Therefore, we can **group** the countries with less representation in a single category (let's call it `Other`).
 
 2) We will also one-hot encode the categorical variables to make them usable in machine learning algorithms. This is possible with the `get_dummies()` method of the Pandas library.
 
 3) Since we have an `age` variable, we can compute age groups using binning. This can be done with the `cut()` method, also available with Pandas.
 

In [162]:
# aggregation - assign value `Other` for countries with less than 200 participants 
min_samples_country = 200
df['native-country-agg'] = df['native-country'].map({x[0]: x[0] if x[1] > min_samples_country else 'Other' for x in dict(df['native-country'].value_counts()).items()})
df.drop(['education', 'native-country'], axis = 1, inplace = True)

# One-hot encoding
cat_cols = []
for col in df.columns:
    if df[col].dtype == 'object':
        # the `dummy_na` parameter adds a separate category for NaN values
        df_dummy = pd.get_dummies(df[col], prefix=col, dummy_na=True)
        df = pd.concat((df, df_dummy), axis = 1 )
        cat_cols.append(col)        

df.drop(cat_cols, axis = 1, inplace = True)

# Binnarization of the age attribute into 5 etary groups (equidistant)
df['age_quantized'] = pd.cut(df['age'], 5, labels = False)
print('Columns after feature extraction & engineering:\n')
print(df.columns)



Columns after feature extraction & engineering:

Index(['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'income', 'workclass_Federal-gov',
       'workclass_Local-gov', 'workclass_Never-worked', 'workclass_Private',
       'workclass_Self-emp-inc', 'workclass_Self-emp-not-inc',
       'workclass_State-gov', 'workclass_Without-pay', 'workclass_nan',
       'marital-status_Divorced', 'marital-status_Married-AF-spouse',
       'marital-status_Married-civ-spouse',
       'marital-status_Married-spouse-absent', 'marital-status_Never-married',
       'marital-status_Separated', 'marital-status_Widowed',
       'marital-status_nan', 'occupation_Adm-clerical',
       'occupation_Armed-Forces', 'occupation_Craft-repair',
       'occupation_Exec-managerial', 'occupation_Farming-fishing',
       'occupation_Handlers-cleaners', 'occupation_Machine-op-inspct',
       'occupation_Other-service', 'occupation_Priv-house-serv',
       'occupation_Prof-specia

Let's take a look at how the `race` attribute was decomposed with the `get_dummies()` method.

In [163]:
df[['race_Amer-Indian-Eskimo','race_Asian-Pac-Islander', 'race_Black', 'race_Other', 'race_White','race_nan',]].head()

,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,race_nan
0,0,0,1,0,0,0
1,0,0,0,0,1,0
2,0,0,0,0,1,0
3,0,0,1,0,0,0
4,0,0,0,0,1,0


Depending on how meany things we try during the feature enginnering step, several new features will be created, which can have a negative impact in model performance and computing time. Therefore, the next step would be to apply some **feature selection** method to reduce this high-dimensional feature space. However, this is the topic of our next, class, so let's leave it at that for now.


## 4.2 - Images

Images are usually represented as three-dimensional arrays of size (W, H, C), where W is the width of the image, H is the height, and C is the color channel. Typically the RGB color space is used, so three channels are present in the C layer. The following sections will discuss basic and advanced techniques for obtaining features (also called descriptors in the computer vision field) from images. All of the methods are availble in Python via the ``cv2`` and ``scipy.ndimage`` packages.

### 4.2.1 - Basic Operations

A typical transformation involves mapping the 3 color channels to a single one that represents the **luminance** (light intensity) of each pixel. Luminance is one of the layers in the YCbCr color space, which is widely used for image and video processing. The figure below shows the example of a colored image and its luminance (grayscaled) version.

<center> <img src="http://www.workwithcolor.com/hh-picture-grayscale-puppy-01.jpg" width=500 /> Color vs luminance (grayscale) image of a cute puppy (<a href=http://www.workwithcolor.com/hh-picture-grayscale-puppy-01.jpg>source</a>).</center>

The benefits of using a single channel is that data is represented with 1/3 of the original size. In addition, the luminance channel is computed using all three RGB layers, so it is correlated to color information as well. Translating RGB images to YCbCr can be easily done using conversions defined in norms like ITU-R BT.601 (see the [Wikipedia](https://en.wikipedia.org/wiki/YCbCr) page for more information).

Another transformation that is commonly applied involves **binarizing** the image, through all the pixels are mappped to value 1 or 0 (also called black&white or foreground-background representation). To compute the binazied version of an image, we can select a threshold value $th$, such that pixels below this threadshold become black, and those above become white. The challenge here is to define the best value for $th$ without losing important information like edges. A common approach to solve this is to use the [Otsu's method](https://en.wikipedia.org/wiki/Otsu%27s_method) for dynamic thresholding. 


<center> <img src = "https://miro.medium.com/max/1144/1*kd097f77QAoRTmuIvu-oUQ.png" width=500 /> Original and binarized version of the Lena image (<a href="https://miro.medium.com/max/1144/1*kd097f77QAoRTmuIvu-oUQ.png">source</a>).</center>
 </img>

Although useful, these operations still result in 2D values, and most of our models work with 1D feature vectors. Of course we could simply flatten the image and work with a 1D vector of pixels, but this could easily scale to thousands of features. Therefore, further preprocessing is required, as will see in the following paragraphs.

### 4.2.2. - Block-based Features

Many image descriptors involve summarizing the amount of color or light intensity in an image. While this might give us important information, we are losing a lot of information in our training step if we compute these values over the entire image. In addition, large scenes contain several regions with different characteristics, making summarization values much less representative of the real data. 

To circumvent this problem, many implementations partition the input image into **patches**, which are smaller blocks extracted from the input image. Each patch can then be used as input for a model that supports 2D data like CNNs. 

For traditional models that deal with 1D feature vectors, we can compute several descriptors for each patch. Examples of commmon image descriptors include: pixel mean and variance, gradient mean and variance, and color layout descriptor.Advanced descriptors include Local Binary Patterns (LBP), DAISY, SURF, etc. All of them can be found in the ``skimage`` package.

 
### 4.2.3 - PCA and Histogram-based Features

Some approaches of image classification involve computing the histograms from descriptors. In this case, each histogram bin is used as an input feature for prediction models. The figure below shows of a solution that uses the Principal Components Analysis (PCA) as descriptor. PCA consists of a dimensionality reduction technique that finds linear combinations of the input features that maximize the explained variance. Each successive linear combination (called components) is orthogonal to the previous ones, i.e. there is no linear correlation between components (it is as if each component explains a portion of our data that was not already explained by the others). The approach of using PCA for face detection is also called **Eigenfaces**, and the name comes from the eigenvectors obtained with the PCA method (the components are actually eigenvectors of the covariance matrix). 


<center> <img src = "https://media.geeksforgeeks.org/wp-content/uploads/20200317125248/eigenfaces.png" width=400/> Eigenfaces of the Labeled Faces in the Wild (LFW) data set (<a href="https://media.geeksforgeeks.org/wp-content/uploads/20200317125248/eigenfaces.png">source</a>).</center>
 </img>

As the figure shows, the Eigenfaces approach works by factorizing different characteristics of the face in each component. The components are usually much smaller in terms of dimensions, so PCA is commonly appplied for the purpose of dimensionaly reduction as well. The main drawback is that PCA instroduces a black-box transformation in our pipeline, so it reduces the explainability of our solution.

A second approach is called **Histogram of Oriented Gradients (HOG)** (paper [here](https://ieeexplore.ieee.org/document/1467360)). This technique involves computing the orientation and the magnitude of gradients in each block of the image, then binning them in histograms with a limited number of directions. 

The HOG algorithm divides the image into smaller blocks of an arbitrary size. For each block, a histogram with $O$ equidistant orientations (tipycally 8) is generated. These histograms can flattened be used as feature vectors like we did with the PCA components. The figure below shows 
<img src = "https://ars.els-cdn.com/content/image/1-s2.0-S1007021411700323-gr1.jpg" width=400> </img>  
<center> Example of a HOG-based feature extraction method (<a href="https://ieeexplore.ieee.org/document/6077989"> source</a>) </center>


### 4.2.4 - Practicing with Python

Let's start by downloading an image dataset called Labaled Faces in the Wild (LFW). It contains images of faces from celebrities like Tony Blair and George W. Bush, as well as their identifiers (definigin a classification problem. The dataset has 5749 classes, but if we limit the faces per person to a minimum of 70, this number is reduced to 7.

In [16]:
# code from: https://scikit-learn.org/stable/auto_examples/applications/plot_face_recognition.html#sphx-glr-auto-examples-applications-plot-face-recognition-py

from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from skimage.feature import hog
import time

############ Data loading #######################
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# fetch_lfw_people() returns an object containing the `images` and a 1D flat version called `data`
n_samples, h, w = lfw_people.images.shape

# for PCA+SVM we use the data directly (as relative pixel positions info is ignored by this model)
X = lfw_people.data
# for HOG+SVM we use the 2D images (HOG will transform them to 1D vectors)
X_img = lfw_people.images

n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("Image size: %dx%d" % (h, w))
print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)


Image size: 50x37
Total dataset size:
n_samples: 1288
n_features: 1850
n_classes: 7


Since our images have a dimensions of 50x37, we start with 1850 features, which is a lot. 

In [20]:
X_scaled = StandardScaler().fit_transform(X)

# Split data and perform a grid search with cross validation
X_train_img, X_test_img, y_train_img, y_test_img = train_test_split(X_scaled, y, test_size=0.25)

# initialize training timer
time_ini = time.time()

clf = GridSearchCV(SVC(kernel="rbf", class_weight="balanced"), param_grid)

# Finding the best SVM with grid search
clf = clf.fit(X_train_img, y_train_img)
print("Best estimator found by grid search:")
print(clf.best_estimator_)

# Predicting the results
y_pred_img = clf.predict(X_test_img)

print("Classification report (Images + SVM):")
print(classification_report(y_test_img, y_pred_img, target_names=target_names))

# end training timer
time_end = time.time()
print(f'Total training time: %.2f seconds' % (time_end - time_ini) )

Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.0001)
Classification report (Images + SVM):
                   precision    recall  f1-score   support

     Ariel Sharon       0.59      0.71      0.65        14
     Colin Powell       0.82      0.84      0.83        55
  Donald Rumsfeld       0.70      0.76      0.73        37
    George W Bush       0.89      0.89      0.89       139
Gerhard Schroeder       0.78      0.90      0.84        20
      Hugo Chavez       0.89      0.76      0.82        21
       Tony Blair       0.97      0.78      0.86        36

         accuracy                           0.84       322
        macro avg       0.81      0.81      0.80       322
     weighted avg       0.85      0.84      0.84       322

Total training time: 177.88 seconds


Our SVM classifier achieved a pretty good score for a problem with 7 classes, with an accuracy of 84%. Our model is an expert at detecting Bush's face, but it is much less efficient with Ariel Sharon. However, the training time was quite lengthy (177.9 *s* is not forever, but this is actually a small-scale data set). If all the pixels are used as features, training could take hours or even days to finish.

Let's reduce this feature space by applying the techninques we discussed: PCA and HOG. We will start with PCA and reduce the amount of features to 160 (no, there's no reason for using this particular number, but its just for demonstration purposes). Note that we must **scale** the values before using the PCA `fit()` method (keep that in mind, as people often forget and get bad results because of it).

In [5]:
print('############ PCA transformation ########################')

# Scaling values is very important for PCA
X_pca = StandardScaler().fit_transform(X)

# Split data and perform a grid search with cross validation
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.25)

n_components = 160
pca = PCA(n_components=n_components).fit(X_train_pca)

# the `pca` object is now fitted and can be used to transform inputs
# X_train_pca will now have 150 features (each component is a feature)
X_train_pca = pca.transform(X_train_pca)
X_test_pca = pca.transform(X_test_pca)
print(f'Number of PCA features {X_train_pca.shape[1]}')


############ PCA transformation ########################
Number of PCA features 160


Now let's see how well our PCA features work when we use them as input for an SVM classifier. You can ignore most of the training steps for now, because we will discuss them in detail later in this course. Just understand that we are doing a model selection to find an efficient SVM fit and provide a fair comparison among alternative methods (we will compare PCA with HOG).

In [18]:
############ Training setup #######################
# Don't worry about this part. We will learn about it during the course

time_ini = time.time()

param_grid = {
    "C": [1e3, 5e3, 1e4, 5e4, 1e5],
    "gamma": [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
}
clf = GridSearchCV(SVC(kernel="rbf", class_weight="balanced"), param_grid)

# Finding the best SVM with grid search
clf = clf.fit(X_train_pca, y_train_pca)
print("Best estimator found by grid search:")
print(clf.best_estimator_)

# Predicting the results
y_pred_pca = clf.predict(X_test_pca)

print("Classification report (PCA + SVM):")
print(classification_report(y_test_pca, y_pred_pca, target_names=target_names))

time_end = time.time()
print(f'Total training time: %.2f seconds' % (time_end - time_ini) )

Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.0001)
Classification report (PCA + SVM):
                   precision    recall  f1-score   support

     Ariel Sharon       0.76      0.72      0.74        18
     Colin Powell       0.77      0.81      0.79        53
  Donald Rumsfeld       0.89      0.75      0.81        32
    George W Bush       0.91      0.93      0.92       144
Gerhard Schroeder       0.70      0.76      0.73        21
      Hugo Chavez       0.92      0.75      0.83        16
       Tony Blair       0.89      0.89      0.89        38

         accuracy                           0.86       322
        macro avg       0.83      0.80      0.82       322
     weighted avg       0.86      0.86      0.86       322

Total training time: 14.83 seconds


The PCA+SVM classifier achieved a pretty good score as well, with an accuracy of 86%. This is actualy a bit a more than the SVM using all the pixels (although it can be considered a technical tie since we should've improved our training parameters). The most important metric here is the training time: we reduced it from 177 *s* to 14.8 *s* (a reduction of more than **90%**). 

Now let's see how our HOG features perform. HOG has several parameters that define the size of the image subblocks, the amount of orientations etc. These parameters have a direct effect at the size of the output (for instance, less blocks lead to less histograms). We will keep most parameters at their default state for now, except for the `pixels_per_cell` parameter. We will set it to 12x12 as a way of getting a similar number of features used in our PCA test.

In [19]:
print('\n############ HOG transformation ########################')
# code from: https://www.kaggle.com/manikg/training-svm-classifier-with-hog-features

# The pixels_per_cell(ppc) parameter must be tuned. We will use 12 because it gives us ~160 features (close to our PCA)
# For more info, see the hog method documentation: https://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.hog
ppc = 12
hog_images = []
hog_features = []
for image in X_img:
    # the feature_vector = True parameter returns the output as a 1D vector
    hog_feat = hog(image, feature_vector = True, pixels_per_cell= (ppc,ppc))
    hog_features.append(hog_feat)


X_hog = np.array(hog_features)
print(f'Number of HOG features {X_hog.shape[1]}')

X_hog = StandardScaler().fit_transform(X_hog)
X_train_hog, X_test_hog, y_train_hog, y_test_hog = train_test_split(X_hog, y, test_size=0.25)

time_ini = time.time()
param_grid = {
    "C": [1e3, 5e3, 1e4, 5e4, 1e5],
    "gamma": [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
}
clf = GridSearchCV(SVC(kernel="rbf", class_weight="balanced"), param_grid)
clf = clf.fit(X_train_hog, y_train_hog)
print("Best estimator found by grid search:")
print(clf.best_estimator_)

y_pred_hog = clf.predict(X_test_hog)

print("Classification report (HOG + SVM):")
print(classification_report(y_test_hog, y_pred_hog, target_names=target_names))

time_end = time.time()
print(f'Total training time: %.2f seconds' % (time_end - time_ini) )


############ HOG transformation ########################
Number of HOG features 162
Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.005)
Classification report (HOG + SVM):
                   precision    recall  f1-score   support

     Ariel Sharon       0.86      0.75      0.80        16
     Colin Powell       0.90      1.00      0.95        64
  Donald Rumsfeld       0.85      0.79      0.81        28
    George W Bush       0.85      0.96      0.91       139
Gerhard Schroeder       1.00      0.61      0.76        31
      Hugo Chavez       1.00      0.50      0.67        16
       Tony Blair       0.81      0.79      0.80        28

         accuracy                           0.87       322
        macro avg       0.90      0.77      0.81       322
     weighted avg       0.88      0.87      0.87       322

Total training time: 11.95 seconds


We can see that the SVM using HOG features presented a similar performance to that of the SVM trained with PCA, with a small gain in computing power. This is not a definitive answer, however, because we did not include the tuning of the HOG/PCA hyperparameters. In any case, these results show that reducing the dimension of our data set with these methods lead to less computing requirements and efficient prediction performance.

## 4.3 - Text

Extracting features from text means we must find a way to map words to values. This can be quite cumbersome to use in machine learning, because documents can have a very large and varied number of words. As we know, our feature vectors must have limited and consistent dimensions across all of our instances.

### 4.3.1 - The Bag of Words model

A widely used approach to solve this is to create Bag of Words (BoW), which is essentially a vector of word counts(similar to a histogram for numeric values). The figure below illustrates the concept of a BoW representation for a given document with raw text.

<img src = "FIGS/4-bag_of_words.png" width = 300> </img>

Note that with the BoW representation our documents (instances) become N-dimensional feature vectors, where N is the number of words in the bag. When we do such mapping, we can start visualizing these sentences in the vector space, enabling interesting operations like computing **distances** (the [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) is commonly used for text). Let's take a look at the illustration below:

<img src = "FIGS/4-bag_of_words_vector_space.png" width = 400> </img>

In this figure, the words "extremely", "cute", and "puppy" are axes (features) in our space, and the position of documents in this space depends on the counts of these words in them. Note that the sentence "it is a puppy and it is extremely cute" is close to both other senteces, since it shares two words with both. At the same time, the two other sentences are more distant from each other because they share only a single word of our BoW.

The BoW model only considers a single word for each value, but we know that the meaning of a word is heavily affected by context. For instance, the word "not" before an adjective inverts its meaning. Therefore, we can improve performance of text-based modeling using combination of two words (called bigrams) or more. We can apply the same Bagging approach and create what is called **Bag of N-grams**, in which every n-gram in our corpus is a feature. Of course this scales very badly, so most applications do not use more than 2 or 3 words.


### 4.3.2 - TF-IDF Scaling

The BoW representation can already be used as input to any machine learning algorithm, as it consistis of 1D feature vectors. However, simply counting the occurrences of a word might not be the best strategy to obtain important information from text documents. For instance, the word "the" occurs many times in English text and will likely reduce the efficiency of text-based applications.

To solve this, we can apply a scaling mechanism that is fit for this type of data (we certainly cannot apply the scalers mentioned in Section 4.1). Fortunately, an efficent technique called **Term Frequency-Inverse Document Frequency** (TF-IDF) was invented. The TF-IDF of each word in our BoW is computed with the following equation: 

<br>
<div style = 'line-height: 3;background-color:#FFF8DC'>
TFIDF$(w, d) =\mathrm{BoW}(w, d) *\log{\frac{N}{D_w}}$,&nbsp;&nbsp;&nbsp; where <br>
$D_w =$ number of documents in which word $w$ appears <br>
BoW$(w, d) =$ times word $w$ appears in document $d$ <br>
$N=$ total number of documents
</div>

The $\log{N/D_w}$ part of the equation represents the IDF term, which is close to $\log{N}$ when a word occurs rarely across documents, and it is close to $0$ when a word occurs too often. In other words, words that are seldom used are upscaled and vice-versa.


## 4.4 - Time Series

Time series can be converted to discrete (quantized) or to numeric data. 

For a discrete conversion, smoothing/aggregation techniques are used. A common approach involves the **SAX (Symbolic Aggregate approXimation)** (paper can be found [here](https://link.springer.com/article/10.1007/s10618-007-0064-z). SAX is built from a previous technique called PAA (Piecewise Aggregate Approximation). The basic idea of PAA is that the time series is segmented, and for each segment the mean value is computed. A more recent approach proposes the 1d-SAX, an improvement of SAX where each segment is represented by an affine function (2 parameters per segment are  quantized, representing the slope and mean value). The figure below shows an illustration of these techniques. The ``tslearn`` python package contains implementations of all these techniques.

<img src = "https://tslearn.readthedocs.io/en/stable/_images/sphx_glr_plot_sax_001.svg" width = 400> </img>

For numeric conversion, two transforms are commonly used: **Discrete Wavelet Transform (DWT)** and **Discrete Fourier Transform (DFT)**. These transforms convert the time series to multidimensional data that can be used in traditional machine learning algorithms. The transformed coefficients can be more easily analyzed because temporal locality is already incorporated in them. In addition, we usually keep a smaller subset of these coefficients, so this approach can be seen as a dimensionality reduction method as well.

Aggarwal suggests that DWT must be applied when the variations in the time series happen in specific local regions of the time series. In cases where the series contain global periodicity, the DFT is more effective. The figure below shows an example of two time series: one that is more suited for DWT, and one that works best with DFT.

<img src="FIGS/4-time_series_dft_dwt.png" width = 400> </img> 

## 4.5 - Summary

- Feature extraction depends a lot on the type of data that is being used
- Many features can only be obtained with the help of domain knowledge from experts, so make sure you have one in your team
- One-hot encoding is useful when nominal variables must be used with numeric methods. For ordinal values, Label Encoding is the way to go.
- Normalizing/scaling your features is a necessary step if distance-based algorithms are used
- Several techniques have been proposed for feature extraction of complex data like images, texts, and time series. We've only covered the very basic ones. Go research some more!

So far we are only worried about creating new features, but this might lead to high-dimensional data that is difficult to work with. In the next lesson, we will discuss feature selection methods that can help us avoid the so-called "Curse of Dimensionality". 👻 

<h1> <center> See you all in our next lesson! &#128516; </center> </h1>